In [3]:
!pip install segmentation_models -q
!pip install albumentations -q

You should consider upgrading via the 'C:\Users\dassu\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.
You should consider upgrading via the 'C:\Users\dassu\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [22]:
import torch
import torch.nn as nn
import torchvision
from torchvision.utils import make_grid, save_image
import torch.optim as optim
import torch.utils.data as datasets
import torchvision.transforms as transforms
import numpy as np
import os
from PIL import Image
import albumentations as albu
from albumentations.pytorch import ToTensorV2
from sklearn.model_selection import train_test_split
import segmentation_models_pytorch as sgm

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [8]:
image_dir = "images"
mask_dir = "masks"

In [13]:
class LungSegmentationDataset(datasets.Dataset):
    def __init__(self, images, transform, image_root="images", mask_root="masks"):
        self.images = images
        self.masks = [img.rstrip(".png")+"_mask.png" if img[0] == "C" else img for img in self.images]
        self.transform = transform

        self.image_root = image_root
        self.mask_root = mask_root

        self.extra_aug = transforms.Compose([
            transforms.ColorJitter(0.2, 0.2, 0.2, 0.2),
            transforms.RandomErasing(p=0.7),
        ])

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = self.images[idx]
        mask_path = self.masks[idx]

        img = Image.open(os.path.join(self.image_root, img_path)).convert("L")
        mask = Image.open(os.path.join(self.mask_root, mask_path)).convert("L")

        data = self.transform(image=np.array(img)/255, mask=np.array(mask)/255)
        img, mask = data["image"], data["mask"]

        img = self.extra_aug(img)

        return img.float(), mask

In [14]:
images = os.listdir(image_dir)
masks = os.listdir(mask_dir)
final_images = []

for path in images:
    if path in masks or path.rstrip(".png")+"_mask.png" in masks:
        final_images.append(path)

train_files, test_files = train_test_split(final_images, train_size=0.8, random_state=17)

In [15]:
train_transforms = albu.Compose([
    albu.Resize(224, 224),
    albu.Affine([0.8, 1.2], [0.1, 0.1], rotate=10, p=0.8),
    ToTensorV2()
], additional_targets={"image": "image", "mask": "image"})

test_transform = albu.Compose([
    albu.Resize(224, 224),
    ToTensorV2()
], additional_targets={"image": "image", "mask": "image"})

In [16]:
batch_size = 16
test_batch_size = 4
lr = 1e-4
epochs = 30

train_data = LungSegmentationDataset(train_files, train_transforms)
test_data = LungSegmentationDataset(test_files, test_transform)

In [17]:
train_loader = datasets.DataLoader(train_data, batch_size, shuffle=True)
test_loader = datasets.DataLoader(test_data, test_batch_size)

model = sgm.Unet(in_channels=1).to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)
criterion = sgm.losses.DiceLoss(sgm.losses.constants.BINARY_MODE)
iou_metric = sgm.losses.JaccardLoss(sgm.losses.constants.BINARY_MODE)

In [18]:
if not os.path.exists("models"):
    os.mkdir("models")

In [19]:
for epoch in range(epochs):
    losses = []
    ious = []
    model.train()

    for i, (img, mask) in enumerate(train_loader):
        optimizer.zero_grad()

        y = model(img.to(device))
        loss = criterion(y, mask.float().to(device))
        ious.append(1-iou_metric(y.detach().cpu(), mask.long()))

        loss.backward()
        optimizer.step()

        losses.append(loss.item())
        print("\r", f"train epoch {epoch}/{epochs}, iter {i}/{len(train_loader)},"
                    f"loss {sum(losses)/(len(losses))}, iou {sum(ious)/len(ious)}", end="")

    print("")
    losses = []
    model.eval()

    for i, (img, mask) in enumerate(test_loader):
        y = model(img.to(device))
        loss = criterion(y, mask.float().to(device))
        ious.append(1-iou_metric(y.detach().cpu(), mask.long()))

        losses.append(loss.item())
        print("\r", f"test epoch {epoch}/{epochs}, iter {i}/{len(test_loader)},"
                    f"loss {sum(losses) / (len(losses))}, iou {sum(ious)/len(ious)}", end="")
    print("")
    torch.save(model, f"models/epoch_{epoch}.pth")

 train epoch 0/30, iter 35/36,loss 0.4823182142443127, iou 0.34877988696098334
 test epoch 0/30, iter 35/36,loss 0.40200303826067185, iou 0.38766023516654975
 train epoch 1/30, iter 35/36,loss 0.3144392536746131, iou 0.513947248458862376
 test epoch 1/30, iter 35/36,loss 0.29825176960892147, iou 0.5272665619850159
 train epoch 2/30, iter 35/36,loss 0.25351491901609635, iou 0.5867607593536377
 test epoch 2/30, iter 35/36,loss 0.23739350669913822, iou 0.6015490293502808
 train epoch 3/30, iter 35/36,loss 0.2082854840490553, iou 0.64659231901168826
 test epoch 3/30, iter 35/36,loss 0.20079750153753492, iou 0.6561807394027719
 train epoch 4/30, iter 35/36,loss 0.17954938610394797, iou 0.6863291263580322
 test epoch 4/30, iter 35/36,loss 0.1780583080318239, iou 0.69188541173934942
 train epoch 5/30, iter 35/36,loss 0.1535012341207928, iou 0.72560507059097296
 test epoch 5/30, iter 35/36,loss 0.15433296892378065, iou 0.7290917634963989
 train epoch 6/30, iter 35/36,loss 0.13930918276309967, 

In [20]:
if not os.path.exists("example_plots"):
    os.mkdir("example_plots")

In [26]:
n_examples = 4
for i in range(n_examples):
    img, mask = next(iter(train_loader))
    preds = torch.sigmoid(model(img.to(device))).cpu()

    pred_grid = make_grid(preds, nrow=4)
    mask_grid = make_grid(mask, nrow=4)

    save_image(pred_grid, f"example_plots/pred_{i}.png")
    save_image(mask_grid, f"example_plots/ground_truth_{i}.png")